In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#SCRAPES AD DATA
from selenium import webdriver
import requests
import requests
import json
from bs4 import BeautifulSoup

def get_info():
    #def fetch_from_538(state):
    forecast_page = "https://projects.fivethirtyeight.com/2020-campaign-ads/#"
    forecast_html = requests.get(forecast_page)
    soup = BeautifulSoup(forecast_html.text, 'html.parser')
    # soup.prettify()
    options = list(soup.find_all('option'))
    #options[0].split("\"")
    options = [str(i).split("\"")[1] for i in options]
    split = len(options)-51
    candidates = [i.lower().replace(" ","-") for i in options if " " in i]
    states = [i.lower() for i in options if " " not in i and i!="" and i!="National"]
    assert len(states)+len(candidates)==len(options)-3
    
    for i in range(len(candidates)): 
        if candidates[i]=='julián-castro':
            candidates[i]='julin-castro'
    return states, candidates


def to_df(name_l, state_l, airings_l, start_l, end_l, cost_l, title_l, lookup_l, issues_l):
    out = pd.DataFrame(data = {"candidate":name_l, "state":state_l, "airings":airings_l, "start":start_l, "end":end_l, "cost":cost_l, "title":title_l, "lookup":lookup_l, "issues":issues_l})
    return out

def parse2(text, starts, ends):
    
    months = ["jan", "feb", "march", "april", "may", "june", "july", "aug", "sept", "oct", "nov", "dec"]

    month_map = {}
    for i in range(len(months)):
        month_map[months[i]]=str(i+1)
    
    words = text.lower().split(" ")
    
    if len(words)==11:
        start_month = month_map[words[4].lower().replace(".","")]
        if len(start_month)==1: start_month = "0"+start_month
        start_day = words[5].replace(",","")
        if len(start_day)==1: start_day = "0"+start_day
        start_year = words[6]
        start_date = start_year+start_month+start_day
        starts.append(start_date)

        end_month = month_map[words[8].lower().replace(".","")]
        if len(end_month)==1: end_month = "0"+end_month
        end_day = words[9].replace(",","")
        if len(end_day)==1: end_day = "0"+end_day
        end_year = words[10]
        end_date = end_year+end_month+end_day
        ends.append(end_date)
    
    elif len(words)==9:
        start_month = month_map[words[4].lower().replace(".","")]
        if len(start_month)==1: start_month = "0"+start_month
        start_day = words[5].replace(",","")
        if len(start_day)==1: start_day = "0"+start_day
        start_year = "2020"
        start_date = start_year+start_month+start_day
        starts.append(start_date)

        end_month = month_map[words[7].lower().replace(".","")]
        if len(end_month)==1: end_month = "0"+end_month
        end_day = words[8].replace(",","")
        if len(end_day)==1: end_day = "0"+end_day
        end_year = "2020"
        end_date = end_year+end_month+end_day
        ends.append(end_date)
        
    elif len(words)==6:
        start_month = month_map[words[4].lower().replace(".","")]
        if len(start_month)==1: start_month = "0"+start_month
        start_day = words[5].replace(",","")
        if len(start_day)==1: start_day = "0"+start_day
        start_year = "2020"
        start_date = start_year+start_month+start_day
        starts.append(start_date)
        ends.append(start_date)
        
    elif len(words)==7:
        start_month = month_map[words[4].lower().replace(".","")]
        if len(start_month)==1: start_month = "0"+start_month
        start_day = words[5].replace(",","")
        if len(start_day)==1: start_day = "0"+start_day
        start_year = "2019"
        start_date = start_year+start_month+start_day
        starts.append(start_date)
        ends.append(start_date)
        
    elif len(words)==10:
        start_month = month_map[words[4].lower().replace(".","")]
        if len(start_month)==1: start_month = "0"+start_month
        start_day = words[5].replace(",","")
        if len(start_day)==1: start_day = "0"+start_day
        start_year = words[6]
        start_date = start_year+start_month+start_day
        starts.append(start_date)

        end_month = month_map[words[8].lower().replace(".","")]
        if len(end_month)==1: end_month = "0"+end_month
        end_day = words[9].replace(",","")
        if len(end_day)==1: end_day = "0"+end_day
        end_year = "2020"
        end_date = end_year+end_month+end_day
        ends.append(end_date)
        
    else:
        print(words)
        assert False

def scrape_ads(fileout="./tvads.csv"):
    name_l = []
    state_l = []
    airings_l = []
    start_l = []
    end_l = []
    cost_l = []
    title_l = []
    lookup_l = []
    issues_l = []

    titles = set()

    states, candidates = get_info()


    chrome_path = "./chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_path)

    for state in states:
        for name in candidates:
            driver.get("https://projects.fivethirtyeight.com/2020-campaign-ads/#candidate="+name+"&market="+state)
            posts = driver.find_elements_by_class_name("ad")
            d = driver.find_elements_by_class_name("date")

            d = [d[i] for i in range(len(d)) if len(d[i].text)>0]



            if len(d) != len(posts):
                print("-----",name,state)
                assert False


            else:
                for j in range(len(posts)):
                    post = posts[j]

                    #print(post.text)
                    title = post.find_elements_by_class_name("title")[0].text
                    if title not in titles:
                        titles.add(title)
                        locs = post.find_elements_by_tag_name("table")[0].find_elements_by_tag_name("tbody")[0].find_elements_by_tag_name("tr")
                        for i in locs: 
                            tokens = i.text.split(" ")
                            if len(tokens)==4:
                                tokens[1]=tokens[0]+" "+tokens[1]
                                tokens=tokens[1:]
                            if len(tokens)!=3:
                                print(tokens)
                                assert False 
                            statea = tokens[0]
                            airings = int(tokens[1].replace(",",""))
                            cost = int(tokens[2][1:].replace(",",""))
                            
                            issues = post.find_elements_by_class_name("issues")
                            if len(issues)==0:
                                print("F:", name,state,title)
                            issues = issues[0].text
                            
        
                            name_l.append(name)
                            lookup_l.append(state)
                            state_l.append(statea)
                            airings_l.append(airings)
                            cost_l.append(cost)
                            title_l.append(title)
                            issues_l.append(issues)
                            
                            parse2(d[j].text, start_l, end_l)        
                            #print(cost, airings, state)
            out = to_df(name_l, state_l, airings_l, start_l, end_l, cost_l, title_l, lookup_l, issues_l)
            print(out)
            out.to_csv(fileout)
            #print(post.find_elements_by_class_name("main")[0].find_elements_by_class_name("markets")[0].find_elements_by_class_name("market text"))
    return out


In [ ]:
scrape_ads()

In [ ]:
#FORMAT DATA
def fix_date(df):
    def to_ymd(x):
        toks = x.split("/")
        for i in range(len(toks)):
            if len(toks[i])==1:
                toks[i]="0"+toks[i]
        return int(toks[2]+toks[0]+toks[1])

    df["date"] = df.modeldate.apply(to_ymd)

def make_names(df):      
    def get_short_name(x):
        if x == 'Joseph R. Biden Jr.': cand = "joe-biden"
        elif x == 'Bernard Sanders': cand = "bernie-sanders"
        else:
            name = x.split(" ")
            cand = name[0].lower()+"-"+name[-1].lower()
        return cand
    df["cand"]=df.candidate_name.apply(get_short_name)
    
    
def get_poll_diffs():
    polls = pd.read_csv("~/Downloads/polls/pres_primary_avgs_2020.csv")
    polls["pct_change"]=np.nan
    make_names(polls)
    fix_date(polls)
    
    polls = polls.sort_values(by=["cand","state","date"])
    polls = polls.reset_index(drop=True)
    
    polls['pct_ch'] = 100*(polls.groupby(["cand","state"])["pct_trend_adjusted"]
                                  .apply(pd.Series.pct_change))
    
    
    polls["log"] = np.log(polls["pct_trend_adjusted"])
    polls['log_ch'] = (polls.groupby(["cand","state"])["log"]
                                  .apply(pd.Series.pct_change))
    return polls
                
    
def get_data(out, polls ,fileout,skip_low=False):
    ev_time_t = []
    state_t = []
    cand_t = []
    poll_t = []
    date_t = []
    title_t = []
    lookup_t = []
    raw_change_t = []
    pct_change_t = []
    log_change_t = []
    daily_pct = []
    daily_log = []
    issues_t = []
    airings_t = []
    cost_t = []
    polla_t = []

    make_names(polls)
    fix_date(polls)
    
    polls = polls.sort_values(by=["cand","state","date"])
    polls = polls.reset_index(drop=True)
    good = 0 

    counter = 0

    for i in out.index:
        counter+=1
        if counter%300==0: 
            print(counter,"/",len(out))
        cand = out.candidate[i]
        state = out.state[i]
        date = out.start[i]
        title = out.title[i]
        lookup = out["lookup"][i]
        iss = out.issues[i]
        arng = out.airings[i]
        cost = out.cost[i]



        row = polls.loc[(polls.state==state)&(polls.cand==cand)&(polls.date==date)]
        

        if len(row)!=0:
            polla = row.pct_trend_adjusted[row.index[0]]
            
            if skip_low:
                assert len(row)==1
                if row.pct_trend_adjusted[row.index[0]]<5:
                    continue
        
            #print("CHECK",cand,state,date,out["lookup"][i],out["title"][i])
            ind = row.index[0]
            lower = ind-30
            upper = ind+30
            
            base = polls.iloc[ind]["pct_trend_adjusted"]

            good+=1

            for j in range(lower,upper):
                curr = polls.iloc[j]
                if curr["cand"]==cand and curr["state"]==state:
                    ev_time_t.append(j-ind)
                    state_t.append(state)
                    cand_t.append(cand)
                    poll_t.append(curr.pct_trend_adjusted)
                    date_t.append(curr["date"])
                    title_t.append(title)
                    lookup_t.append(lookup)
                    raw_change_t.append(curr.pct_trend_adjusted-base)
                    pct_change_t.append((curr.pct_trend_adjusted-base)/base)
                    log_change_t.append(np.log(curr.pct_trend_adjusted)-np.log(base))
                    daily_pct.append(curr.pct_ch)
                    issues_t.append(iss)
                    daily_log.append(curr.log_ch)
                    airings_t.append(arng)
                    cost_t.append(cost)
                    polla_t.append(polla)
                    
        else:
            assert len(row)==0

    data = pd.DataFrame(data = {"ev_time": ev_time_t, "state": state_t, "cand": cand_t, "poll" : poll_t, "date":date_t, "title":title_t, "lookup":lookup_t, "raw_change":raw_change_t, "pct_change":pct_change_t, "log_change":log_change_t, "daily_pct":daily_pct, "daily_log":daily_log, "issues":issues_t, "airings":airings_t, "cost":cost_t, "polla":polla_t})    
    data.to_csv(fileout)
    return data

In [ ]:
out = pd.read_csv("./tvads.csv")
polls = get_poll_diffs()
data = get_data(out, polls, "./data.csv")

In [ ]:
#ADD FEATURE COLUMNS
def quantiles(data,n=5,pref="q"):
    qs = [round(1/n*i*100) for i in range(n+1)]
    print(qs)
    for i in range(1,len(qs)):
        lb = np.percentile(data.loc[data.polla.isnull()==False, "polla"],qs[i-1])
        ub = np.percentile(data.loc[data.polla.isnull()==False, "polla"],qs[i])
        print((lb,ub))
        data[pref+"tile_"+str(i)]=0
        data.loc[(data.polla>lb)&(data.polla<=ub),pref+"tile_"+str(i)]=1

def lb(data):
    lbs = [i for i in range(0,16,3)][1:]
    for i in lbs:
        data["lb_"+str(i)]=0
        data.loc[data.polla>i, "lb_"+str(i)]=1

def candstate(data):
    for c in data.cand.unique():
        for s in data.state.unique():
            data["candstate_"+c+"_"+s]=0
            data.loc[(data.cand==c)&(data.state==s),"candstate_"+c+"_"+s]=1
            
    allct = [i for i in data.columns if i[:len("candstate")]=="candstate"]
    dropct = [i for i in allct if len(data[i].unique())==1]
    data = data.drop(columns = dropct)
    return data

def candtime(data):
    hmon = ["00","15"]
    data = add_time_controls(data,"hmon",hmon)
    hmons = [i for i in data.columns if i[:4]=="hmon"]
    
    for c in data.cand.unique():
        for hm in hmons:
            data["candtime_"+c+"_"+hm]=0
            data.loc[(data.cand==c)&(data[hm]==1),"candtime_"+c+"_"+hm]=1
            
    allct = [i for i in data.columns if i[:len("candtime")]=="candtime"]
    dropct = [i for i in allct if len(data[i].unique())==1]
    data = data.drop(columns = dropct)
    return data
    

def topic_cols(data, group):
    cols = [i for i in data.columns if i[:4]=="iss_"]
    iss = [i[4:] for i in data.columns if i[:4]=="iss_"]
    counts = []
    for i in cols:
        counts.append(data.loc[data.ev_time==0,i].sum())
    
    sorted_iss = sorted([(iss[i],counts[i],cols[i]) for i in range(len(iss))], key = lambda x: x[1], reverse = True)
    iss = [i[0] for i in sorted_iss]
    counts = [i[1] for i in sorted_iss]
    cols = [i[2] for i in sorted_iss]

    for ind in range(len(cols)):
        for i in range(data[group].max()+1):
            data["topic_"+iss[ind]+"_"+str(i)]=0
            data.loc[(data[group]==i)&(data[cols[ind]]==1),"topic_"+iss[ind]+"_"+str(i)]=1

def gen_color(data):
    el = pd.read_csv("elections.csv")
    el["party"]="r"
    el.loc[el.dem>el.rep,"party"]="b"
    el.loc[np.abs(el.dem-el.rep)<=5,"party"]="p"

    color_map = dict([(el.iloc[i].full,el.iloc[i].party) for i in el.index])
    color_map["National"]=np.nan

    data["color"]="r"
    for s in data.state.unique():
        data.loc[data.state==s,"color"]=color_map[s]


def median_split(data, col):
    med = data.loc[data.ev_time==0, col].median()
    data["med_"+col]=0
    data.loc[data[col]>=med,"med_"+col]=1

def make_issue_cols(data):
    print(len(data.issues.unique()))
    data = data.drop(data.loc[data["issues"].isnull()].index)
    issues = ";".join(data.issues.unique()).replace(" ","").split(";")

    issues = list(set(issues))

    for i in issues:
        data["iss_"+i.lower()] = 0

    for i in data.index:
        iss = data["issues"][i]
        l = iss.replace(" ","").split(";")
        for j in l:
            data["iss_"+j.lower()][i] = 1
            
    return data
    

def discretize_ev_time(data,num):
    def threes(x):
        if x==0: return 0
        if x<0: return x//num
        if x>0: return -1*((-1*x)//num)

    data["grouped_"+str(num)] = data.ev_time.apply(threes)

def add_state_controls(data):
    for s in data.state.unique():
        name = s.replace(" ","").lower()
        data["in_"+name]=0
        data.loc[data.state==s,"in_"+name]=1
        
def add_time_controls(df,pref, days):
    def add_zero(i):
        s = str(i)
        if len(s)==1: s="0"+s
        return s

    last = 0

    for y in ["2019","2020"]:
        for m in [add_zero(i) for i in range(1,13)]:
            for d in days:
                thresh = int(y+m+d)
                df[pref+"_"+str(thresh)]=0
                df.loc[(df.date<=thresh) & (df.date>last),pref+"_"+str(thresh)] = 1
                last = thresh
    
    to_drop = []
    for c in df.columns:
        if c[:len(pref)]==pref and 1 not in df[c].unique():
            to_drop.append(c)
    print(to_drop)
    df = df.drop(columns = to_drop)
    return df


def get_diffs(df):
    df["pct_daily"] = np.nan
    for i in df.index[1:]:
        state = df.state[i]
        cand = df.cand[i]
        title = df.title[i]
        
        l_state = df.state[i-1]
        l_cand = df.cand[i-1]
        l_title = df.title[i-1]
        
        if state==l_state and cand==l_cand and title==l_title:
            assert df.ev_time[i]==df.ev_time[i-1]+1
            #pct_daily.append((df.poll[i]-df.poll[i-1])/df.poll[i-1])
            df["pct_daily"][i]=(df.poll[i]-df.poll[i-1])/df.poll[i-1]

def drop_bad(data):
    data = data.drop(data.loc[data["pct_change"]==np.inf].index)
    data = data.drop(data.loc[data["log_change"]==np.inf].index)
    data = data.drop(data.loc[data["log_change"]==-np.inf].index)
    data = data.drop(data.loc[data["daily_pct"]==np.inf].index)
    data = data.drop(data.loc[data["daily_log"]==np.inf].index)
    return data

def drop_steyer(data):
    data = data.drop(data.loc[data["cand"]=="tom-steyer"].index)
    return data


In [ ]:
data = pd.read_csv("./data.csv")

data = drop_steyer(data)

discretize_ev_time(data,3)
discretize_ev_time(data,4)
discretize_ev_time(data,5)
discretize_ev_time(data,6)
discretize_ev_time(data,7)
discretize_ev_time(data,10)
discretize_ev_time(data,6)

add_state_controls(data)
data = drop_bad(data)

data = make_issue_cols(data)

topic_cols(data,"grouped_6")

median_split(data, "airings")
median_split(data, "cost")
median_split(data, "poll")

gen_color(data)

quantiles(data,n=4,pref="f")
quantiles(data,n=5,pref="q")
quantiles(data,n=6,pref="s")

data = candtime(data)

week = ["00","07","14","21","28"]
data = add_time_controls(data,"week",week)

week = ['00', '03', '06', '09', '12', '15', '18', '21', '24', '27']
data = add_time_controls(data,"hweek",week)

data.to_csv("./formatted_data.csv")

In [ ]:
#GENERATE PLACEBO DATA
date = 20200303

st = out.loc[out.state.isin(super_tues)& (out.start<=date)]

def mode(x):
    a = x.value_counts()
    if len(a)==0: print(x)
    return a.index[0]

def other_states(x):
    super_tues = ["Alabama","Arkansas","California","Colorado","Maine","Massachusettes","Minnesota","North Carolina","Oklahoma","Tennessee","Texas","Utah","Vermont","Virginia"]
    super_tues = set(super_tues)
    return super_tues-set(x)
    

st.loc[st.issues.isnull(),"issues"]="na"

dictdf = st.groupby(by=["title"]).agg({"candidate":mode,
                                      "airings":"mean",
                                      "start":mode,
                                      "end":mode,
                                      "cost":"mean",
                                      "lookup":mode,
                                      "issues":mode})

other_states_df = st.groupby(by="title")["state"].apply(other_states)
assert len(other_states_df.index)==len(other_states_df.index.unique())
os_dict = other_states_df.to_dict()


candl = []
airingsl = []
startl = []
endl = []
costl = []
lookupl = []
issuesl = []
statel = []
titlel = []

for i in dictdf.index:
    for other_state in os_dict[i]:
        candl.append(dictdf.candidate[i])
        airingsl.append(dictdf.airings[i])
        startl.append(dictdf.start[i])
        endl.append(dictdf.end[i])
        costl.append(dictdf.cost[i])
        lookupl.append(dictdf['lookup'][i])
        issuesl.append(dictdf.issues[i])
        statel.append(other_state)
        titlel.append(i)
        
placebo_out = pd.DataFrame({'candidate':candl, 'state':statel, 'airings':airingsl, 'start':startl, 'end':endl, 'cost':costl,
       'title':titlel, 'lookup':lookupl, 'issues':issuesl})

polls = get_poll_diffs()
placebo_data = get_data(placebo_out, polls, "~/Downloads/cdtest/data_placebo_actual.csv")


In [ ]:
#FORMAT PLACEBO DATA

data = pd.read_csv("~/Downloads/cdtest/data_placebo_actual.csv")
#data = pd.read_csv("~/Downloads/cdtest/data_home_try_candtime_skip.csv")

data = drop_steyer(data)

discretize_ev_time(data,3)
discretize_ev_time(data,4)
discretize_ev_time(data,5)
discretize_ev_time(data,6)
discretize_ev_time(data,7)
discretize_ev_time(data,10)
discretize_ev_time(data,6)
#discretize_ev_time(data,7)
#discretize_ev_time(data,8)
add_state_controls(data)
data = drop_bad(data)

data = make_issue_cols(data)

#topic_cols(data,"grouped_10")
#topic_cols(data,"grouped_7")
topic_cols(data,"grouped_6")


median_split(data, "airings")
median_split(data, "cost")
median_split(data, "poll")

gen_color(data)

#lb(data)
quantiles(data,n=4,pref="f")
quantiles(data,n=5,pref="q")
quantiles(data,n=6,pref="s")

data = candtime(data)
#data = candstate(data)

#data.loc[data.ev_time<=0,"polla"]=0


week = ["00","07","14","21","28"]
data = add_time_controls(data,"week",week)

week = ['00', '03', '06', '09', '12', '15', '18', '21', '24', '27']
data = add_time_controls(data,"hweek",week)

data.to_csv("./placebo_data.csv")

In [ ]:
#GENERATE REAL DATA FOR PLACEBO TEST
date = 20200303

non_placebo = out.loc[out.state.isin(super_tues)& (out.start<=date)].copy()

polls = get_poll_diffs()
nonplacebo_data = get_data(non_placebo, polls, "~/Downloads/cdtest/data_nonplacebo_actual.csv")

In [ ]:
#FORMAT REAL DATA FOR PLACEBO TEST

data = pd.read_csv("~/Downloads/cdtest/data_nonplacebo_actual.csv")

data = drop_steyer(data)

discretize_ev_time(data,3)
discretize_ev_time(data,4)
discretize_ev_time(data,5)
discretize_ev_time(data,6)
discretize_ev_time(data,7)
discretize_ev_time(data,10)
discretize_ev_time(data,6)
add_state_controls(data)
data = drop_bad(data)

data = make_issue_cols(data)

topic_cols(data,"grouped_6")


median_split(data, "airings")
median_split(data, "cost")
median_split(data, "poll")

gen_color(data)

quantiles(data,n=4,pref="f")
quantiles(data,n=5,pref="q")
quantiles(data,n=6,pref="s")

data = candtime(data)

week = ["00","07","14","21","28"]
data = add_time_controls(data,"week",week)

week = ['00', '03', '06', '09', '12', '15', '18', '21', '24', '27']
data = add_time_controls(data,"hweek",week)

data.to_csv("./real_data.csv")